# Pauta Tarea 2

- Usando las funciones anteriores, o desarrollando las propias, escriba una función que pueda valorizar, por simulación de Montecarlo, calls y puts sobre bonos cero cupón con el modelo de HW. Considere puts y calls de plazo máximo igual a 1Y.
- Su función debe poder recibir la semilla.
- Compare sus resultados con las fórmulas del modelo y muestre que diferencias obtiene usando 1000 simulaciones y una semilla predeterminada.
- Valorice también por simulación de Montecarlo el bono cupón cero a 1Y. Muestre que diferencias obtiene respecto al valor de mercado del bono usando 1000 simulaciones y una semilla predeterminada (valor de mercado significa el valor que hoy se puede observar .

## Configuración

### Librerías

In [1]:
import sys
sys.path.insert(1, '../modules')
import auxiliary as aux

import hull_white as hw
import auxiliary as aux
from typing import Callable
import pandas as pd
import numpy as np
import math

### Formatos para DataFrames

In [2]:
frmt = {'plazo': '{:,.0f}', 'tasa': '{:.8%}', 'df': '{:.6%}', 't': '{:.4f}',
        'analytic_put': '{:.6%}', 'sim_put': '{:.6%}', 'dif_put': '{:.6%}',
        'analytic_call': '{:.6%}', 'sim_call':  '{:.6%}', 'dif_call': '{:.6%}',
        'analytic_zero': '{:.6%}', 'sim_zero':  '{:.6%}', 'dif_zero': '{:.6%}', }

### Constante `DT`

In [3]:
DT = 1 / 264.0

### Curva Cero Cupón

In [4]:
df_curva = pd.read_excel('../data/20201012_built_sofr_zero.xlsx')
df_curva['t'] = df_curva['plazo'] / 365.0

In [5]:
df_curva.tail().style.format(frmt)

,plazo,tasa,df,t
28,"7,306",0.92532411%,83.092401%,20.0164
29,"9,133",0.96571892%,78.533775%,25.0219
30,"10,957",0.98810273%,74.332619%,30.0192
31,"14,610",0.94083244%,68.619685%,40.0274
32,"18,262",0.84642634%,65.475678%,50.0329


### Funciones del Modelo HW

- `zrate`: es el cubic spline construido a partir de las columnas `t` y `tasa` de `df_curva`.
- `hwz`: es una versión más sencilla de la función `zero_hw` que sólo requiere llos parámetros `r`, `t` y `T`.
- `theta`: es la misma función que ya utilizamos.

Primero se establecen los valores de $\gamma$ y $\sigma$.

In [6]:
gamma = 1
sigma = .005

Se construyen las funciones `zrate`, `hwz` y `theta`.

In [7]:
zrate, hwz, theta = hw.get_zrate_hwzero_and_theta(
    df_curva['t'],
    df_curva['tasa'],
    gamma,
    sigma
)

## Función para Valorizar

Primero se define la función `payoff` para las call y put sobre un bono cupón cero.

In [8]:
def call_put_payoff(
    c_p: hw.CallPut,
    strike: float,
    t: float,
    T: float,
    r: float,
    hwz: Callable[[float, float, float], float]) -> float:
    """
    """
    if c_p == hw.CallPut.CALL:
        return max(hwz(r, t, T) - strike, 0.0)
    else:
        return max(strike - hwz(r, t, T), 0.0)

Se fijan los parámetros `c_p`, `strike`, `t`, `T` y `hwz`.

In [9]:
def get_call_put_payoff(
    c_p: hw.CallPut,
    strike: float,
    t: float,
    T: float,
    hwz: Callable[[float, float, float], float]) -> Callable[[float,], float]:
    """
    """
    def payoff(r: float):
        return call_put_payoff(c_p, strike, t, T, r, hwz)
    return payoff

Se fijan los parámetros de simulación de la función `hw.sim_hw_many`.

In [10]:
def get_simulator(
    gamma: float,
    sigma: float,
    theta: Callable[[float,], float],
    num_sim: int,
    num_steps: int,
    r0: float,
    seed=None):
    """
    """
    def simulator(seed=None):
        return hw.sim_hw_many(gamma, sigma, theta, r0, num_sim, num_steps, seed)
    return simulator

Se definen `payoff` y `simulator`.

In [11]:
call = get_call_put_payoff(hw.CallPut.CALL, .99, 1, 2, hwz)

In [12]:
simulator = get_simulator(gamma, sigma, theta, 100, 264, zrate(.000001), 16127)

Finalmente, con estos últimos dos elementos se define la función de valorización:

In [13]:
def value_payoff_sim(simulation, payoff) -> float:
    """
    La variable `simulation` son todas las simulaciones de la tasa corta para un `t` predeterminado
    (corte transversal.)
    """
    result = 0.0
    for s in simulation:
        result += math.exp(-DT * np.sum(s)) * payoff(s[-1])
    return result / len(simulation)

In [14]:
time, simulation = simulator()

In [15]:
value = value_payoff_sim(simulation, call)
print(f'simulated value: {value:.6%}')

simulated value: 0.959423%


## Diferencias con Calls y Puts

- Se muestra las diferencias para Calls y Puts y strikes desde 98% hasta 102% en intervalos de 0.5%.
- Los subyacentes son bonos cupón cero de 2Y, 3Y, 4Y y 5Y.
- Todas las opciones tienen vencimiento en 1Y.

In [16]:
compare = []
simulator = get_simulator(gamma, sigma, theta, 100, 264, zrate(.000001), 16127)
time, simulation = simulator()
for T in [2, 3, 4, 5]:
    for strike in [.980, .985, .990, 1.000, 1.005, 1.010, 1.015, 1.020]:
        zo = math.exp(-zrate(1))
        zb = math.exp(-zrate(T) * T)
        
        analytic_call = hw.zcb_call_put(hw.CallPut.CALL, strike, 1, T, zrate(.000001), zo, zb, gamma, sigma)
        call = get_call_put_payoff(hw.CallPut.CALL, strike, 1, T, hwz)
        sim_call = value_payoff_sim(simulation, call)
        
        analytic_put = hw.zcb_call_put(hw.CallPut.PUT, strike, 1, T, zrate(.000001), zo, zb, gamma, sigma)
        put = get_call_put_payoff(hw.CallPut.PUT, strike, 1, T, hwz)
        sim_put = value_payoff_sim(simulation, put)
        
        compare.append((T, strike,
                        analytic_call, sim_call, analytic_call - sim_call,
                        analytic_put, sim_put, analytic_put - sim_put))

df_compare = pd.DataFrame(compare)
df_compare.columns = ['T', 'strike',
                      'analytic_call', 'sim_call', 'dif_call',
                      'analytic_put', 'sim_put', 'dif_put']

In [17]:
df_compare.style.format(frmt)

,T,strike,analytic_call,sim_call,dif_call,analytic_put,sim_put,dif_put
0,2,0.980000,1.955358%,1.962252%,-0.006895%,0.000000%,0.000000%,0.000000%
1,2,0.985000,1.455709%,1.462521%,-0.006812%,0.000000%,0.000000%,0.000000%
2,2,0.990000,0.956060%,0.962789%,-0.006730%,0.000000%,0.000000%,0.000000%
3,2,1.000000,0.063001%,0.067601%,-0.004600%,0.106239%,0.104275%,0.001965%
4,2,1.005000,0.000297%,0.000000%,0.000297%,0.543184%,0.536405%,0.006779%
5,2,1.010000,0.000000%,0.000000%,0.000000%,1.042536%,1.036136%,0.006400%
6,2,1.015000,0.000000%,0.000000%,0.000000%,1.542185%,1.535868%,0.006317%
7,2,1.020000,0.000000%,0.000000%,0.000000%,2.041834%,2.035599%,0.006235%
8,3,0.980000,1.844658%,1.853942%,-0.009284%,0.000000%,0.000000%,0.000000%
9,3,0.985000,1.345009%,1.354211%,-0.009202%,0.000000%,0.000000%,0.000000%


## Diferencias con Bonos Cupón Cero

In [18]:
def zcb(r: float):
    return 1

In [19]:
simulator = get_simulator(gamma, sigma, theta, 1000, 20 * 264, zrate(.000001), 16127)
time, simulation = simulator()

In [20]:
compare2 = []
for T in [1, 2, 3, 4, 5, 10, 12, 15, 20]:
    analytic_zero = math.exp(-zrate(T) * T)
    stop_index = list(time).index(T)
    simulazione = simulation[:, 0:stop_index]
    sim_zero = value_payoff_sim(simulazione, zcb)

    compare2.append((T, analytic_zero, sim_zero, analytic_zero - sim_zero))

df_compare2 = pd.DataFrame(compare2)
df_compare2.columns = ['T', 'analytic_zero', 'sim_zero', 'dif_zero']

In [21]:
df_compare2.style.format(frmt)

,T,analytic_zero,sim_zero,dif_zero
0,1,99.929787%,99.928893%,0.000894%
1,2,99.886549%,99.880187%,0.006362%
2,3,99.775849%,99.760896%,0.014953%
3,4,99.499571%,99.473609%,0.025962%
4,5,99.036413%,98.997534%,0.038879%
5,10,94.359063%,94.312132%,0.046931%
6,12,92.038455%,91.979683%,0.058772%
7,15,88.612207%,88.521547%,0.090660%
8,20,83.109040%,83.044233%,0.064807%


In [23]:
hw.zcb_call_put(
    hw.CallPut.CALL,
    strike=.9,
    to=1,
    tb=2,
    r0=zrate(0.0001),
    zo=math.exp(-zrate(1)),
    zb=math.exp(-zrate(2)*2),
    gamma=.5,
    sigma=.015)

0.09949740530631479